## Imports and constants

In [ ]:
import numpy as np
import matplotlib
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
from cycler import cycler
from copy import copy
matplotlib.rcParams.update({'figure.autolayout': True})
pd.set_option('mode.chained_assignment',None)

In [ ]:
import re
dateFormatExcel = re.compile('\d{2}/\d{2}/\d{4}')
dateFormatOk = re.compile('\d{2}-\d{2}-\d{2}')

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

In [ ]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Range1d
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10
from bokeh.layouts import gridplot

In [ ]:
# %matplotlib inline
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
# prop_cycle = plt.rcParams['axes.prop_cycle']
# colors = prop_cycle.by_key()['color']
new_color_cycle = cycler(color=['#ff7f0e', '#1f77b4', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
plt.rcParams['axes.prop_cycle'] = new_color_cycle

In [ ]:
mainDir = "C://Users//JosephVermeil//Desktop//ActinCortexAnalysis"
experimentalDataDir = os.path.join(mainDir, "ExperimentalData")
dataDir = os.path.join(mainDir, "DataAnalysis")
timeSeriesDataDir = os.path.join(dataDir, "TimeSeriesData")
allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
allTimeSeriesDataFiles

In [ ]:
def get_R2(Y1, Y2):
    meanY = np.mean(Y1)
    meanYarray = meanY*np.ones(len(Y1))
    SST = np.sum((Y1-meanYarray)**2)
    SSE = np.sum((Y2-meanYarray)**2)
    R2 = SSE/SST
    return(R2)

def getDictAggMean(df):
    dictAggMean = {}
    for c in df.columns:
#         t = df[c].dtype
#         print(c, t)
        try :
            if np.array_equal(df[c], df[c].astype(bool)):
                dictAggMean[c] = 'first'
            else:
                try:
                    np.mean(df[c])
                    dictAggMean[c] = 'mean'
                except:
                    dictAggMean[c] = 'first'
        except:
                dictAggMean[c] = 'first'
    return(dictAggMean)

## TimeSeries functions

In [ ]:
def getCellTimeSeriesData(cellID):
    allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
    fileFound = False
    nFile = len(allTimeSeriesDataFiles)
    iFile = 0
    while (not fileFound) and (iFile < nFile):
        f = allTimeSeriesDataFiles[iFile]
        if f.startswith(cellID):
            timeSeriesDataFilePath = os.path.join(timeSeriesDataDir, f)
            timeSeriesDataFrame = pd.read_csv(timeSeriesDataFilePath, sep=';')
            fileFound = True
        iFile += 1
    if not fileFound:
        timeSeriesDataFrame = pd.DataFrame([])
    return(timeSeriesDataFrame)

def plotCellTimeSeriesData(cellID):
    X = 'T'
    Y = np.array(['B', 'F', 'dx', 'dy', 'dz', 'D2', 'D3'])
    units = np.array([' (mT)', ' (pN)', ' (µm)', ' (µm)', ' (µm)', ' (µm)', ' (µm)'])
    timeSeriesDataFrame = getCellTimeSeriesData(cellID)
    if not timeSeriesDataFrame.size == 0:
#         plt.tight_layout()
#         fig.show() # figsize=(20,20)
        axes = timeSeriesDataFrame.plot(x=X, y=Y, kind='line', ax=None, subplots=True, sharex=True, sharey=False, layout=None, \
                       figsize=(8,10), use_index=True, title = cellID + '- Time dependant data', grid=None, legend=False, style=None, logx=False, logy=False, \
                       loglog=False, xticks=None, yticks=None, xlim=None, ylim=None, rot=None, fontsize=None, colormap=None, \
                       table=False, yerr=None, xerr=None, secondary_y=False, sort_columns=False)
        plt.gcf().tight_layout()
        for i in range(len(Y)):
            axes[i].set_ylabel(Y[i] + units[i])
        
    else:
        print('cell not found')
        
def addExcludedCell(cellID, motive):
    f = open(os.path.join(experimentalDataDir, 'ExcludedCells.txt'), 'r')
    lines = f.readlines()
    nLines = len(lines)
    excludedCellsList = []
    for iLine in range(nLines):
        line = lines[iLine]
        splitLine = line[:-1].split(',')
        excludedCellsList.append(splitLine[0])
    if cellID in excludedCellsList:
        newlines = copy(lines)
        iLineOfInterest = excludedCellsList.index(cellID)
        if motive not in newlines[iLineOfInterest][:-1].split(','):
            newlines[iLineOfInterest] = newlines[iLineOfInterest][:-1] + ',' + motive + '\n'            
    else:
        newlines = copy(lines)
        newlines.append('' + cellID + ',' + motive + '\n')
    f.close()
    f = open(os.path.join(experimentalDataDir, 'ExcludedCells.txt'), 'w')
    f.writelines(newlines)
    
def getExcludedCells():
    f = open(os.path.join(experimentalDataDir, 'ExcludedCells.txt'), 'r')
    lines = f.readlines()
    nLines = len(lines)
    excludedCellsDict = {}
    for iLine in range(nLines):
        line = lines[iLine]
        splitLine = line[:-1].split(',')
        excludedCellsDict[splitLine[0]] = splitLine[1:]
    return(excludedCellsDict)

In [ ]:
df = getCellTimeSeriesData('21-02-10_M1_P1_C1')
plotCellTimeSeriesData('21-02-10_M2_P1_C4')

In [ ]:
plotCellTimeSeriesData('21-02-10_M1_P1_C2')

In [ ]:
# Test of the get R2 function.
# T = df['T']
# Y = df['D3']
# plt.plot(T,Y)
# p, residuals, rank, singular_values, rcond = np.polyfit(T, Y, deg=5, full=True)
# plt.plot(T, Y)
# Y2 = np.zeros(len(T))
# for i in range(len(T)):
#     deg = len(p)-1
#     for k in range(deg+1):
#         Y2[i] += p[k]*(T[i]**(deg-k))
# plt.plot(T,Y2)
# get_R2(Y, Y2)

## GlobalTables functions

### Experimental conditions

In [ ]:
def getExperimentalConditions(save = False):
    # Getting the table
    experimentalDataFile = 'ExperimentalConditions.csv'
    experimentalDataFilePath = os.path.join(experimentalDataDir, experimentalDataFile)
    expConditionsDF = pd.read_csv(experimentalDataFilePath, sep=';',header=0)
    print('Extracted a table with ' + str(expConditionsDF.shape[0]) + ' lines and ' + str(expConditionsDF.shape[1]) + ' columns.')
    # Cleaning the table
    try:
        for c in expConditionsDF.columns:
            if 'Unnamed' in c:
                expConditionsDF = expConditionsDF.drop([c], axis=1)
        expConditionsDF = expConditionsDF.convert_dtypes()

        listTextColumns = []
        for col in expConditionsDF.columns:
            if expConditionsDF[col].dtype == 'string':
                listTextColumns.append(col)

        expConditionsDF[listTextColumns] = expConditionsDF[listTextColumns].apply(lambda x: x.str.replace(',','.'))

        expConditionsDF['scale pixel per um'] = expConditionsDF['scale pixel per um'].astype(float)
        expConditionsDF['optical index correction'] = \
                  expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[0]).astype(float) \
                / expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[1]).astype(float)
        expConditionsDF['magnetic field correction'] = expConditionsDF['magnetic field correction'].astype(float)
        expConditionsDF['with fluo images'] = expConditionsDF['with fluo images'].astype(bool)

        expConditionsDF['ramp field'] = \
        expConditionsDF['ramp field'].apply(lambda x: [x.split(';')[0], x.split(';')[1]] if not pd.isnull(x) else [])


        dateExemple = expConditionsDF.loc[expConditionsDF.index[1],'date']

        if re.match(dateFormatExcel, dateExemple):
            print('dates corrected')
            expConditionsDF.loc[1:,'date'] = expConditionsDF.loc[1:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])        
        
    except:
        print('Unexpected bug with the cleaning step')

    if save:
        saveName = 'ExperimentalConditions.csv'
        savePath = os.path.join(experimentalDataDir, saveName)
        expConditionsDF.to_csv(savePath, sep=';')

    expConditionsDF['manipID'] = expConditionsDF['date'] + '_' + expConditionsDF['manip']
    
    return(expConditionsDF)

In [ ]:
getExperimentalConditions(save=False).head()

### Constant Field

In [ ]:
def analyseTimeSeries_ctField(tS_df):
    results = {}
    results['duration'] = np.max(tS_df['T'])
    results['medianRawB'] = np.median(tS_df.B)
    results['medianThickness'] = np.median(tS_df.D3)
    results['1stDThickness'] = np.percentile(tS_df.D3, 10)
    results['9thDThickness'] = np.percentile(tS_df.D3, 90)
    results['fluctuAmpli'] = results['9thDThickness'] - results['1stDThickness']
    results['validated'] = (results['1stDThickness'] > 0)
    X, Y = tS_df['T'], tS_df['D3']
    p, residuals, rank, singular_values, rcond = np.polyfit(X, Y, deg=5, full=True)
    Y2 = np.zeros(len(X))
    for i in range(len(X)):
        deg = len(p)-1
        for k in range(deg+1):
            Y2[i] += p[k]*(X[i]**(deg-k))
    results['R2_polyFit'] = get_R2(Y, Y2)
    return(results)

def createCtFieldDataDict(list_ctFieldFiles):
    tableDict = {}
    tableDict['date'], tableDict['cellName'], tableDict['cellID'], tableDict['manipID'] = [], [], [], []
    tableDict['duration'], tableDict['medianRawB'], tableDict['medianThickness'] = [], [], []
    tableDict['1stDThickness'], tableDict['9thDThickness'], tableDict['fluctuAmpli'] = [], [], []
    tableDict['R2_polyFit'], tableDict['validated'] = [], []
    for f in list_ctFieldFiles:
        split_f = f.split('_')
        tableDict['date'].append(split_f[0])
        tableDict['cellName'].append(split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        tableDict['cellID'].append(split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        tableDict['manipID'].append(split_f[0] + '_' + split_f[1])
        tS_DataFilePath = os.path.join(timeSeriesDataDir, f)
        current_tS_df = pd.read_csv(tS_DataFilePath, ';')
        current_resultDict = analyseTimeSeries_ctField(current_tS_df)
        for k in current_resultDict.keys():
            tableDict[k].append(current_resultDict[k])
    return(tableDict)

def computeGlobalTable_ctField(task = 'fromScratch', fileName = 'Global_CtFieldData', save = False):
    ctFieldTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) \
                                  if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv") \
                                      and ('thickness' in f))]
#     print(ctFieldTimeSeriesDataFiles)
    if task == 'fromScratch':
        # create a dict containing the data
        tableDict = createCtFieldDataDict(ctFieldTimeSeriesDataFiles)
        # create the table
        CtField_DF = pd.DataFrame(tableDict)
        
    elif task == 'updateExisting':
        # get existing table
        try:
            savePath = os.path.join(dataDir, (fileName + '.csv'))
            existing_CtField_DF = pd.read_csv(savePath, sep=';')
            for c in existing_CtField_DF.columns:
                if 'Unnamed' in c:
                    existing_CtField_DF = existing_CtField_DF.drop([c], axis=1)
        except:
            print('No existing table found')
        # find which of the time series files are new
        new_ctFieldTimeSeriesDataFiles = []
        for f in ctFieldTimeSeriesDataFiles:
            split_f = f.split('_')
            currentCellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
            if currentCellID not in existing_CtField_DF.cellID.values:
                new_ctFieldTimeSeriesDataFiles.append(f)
        new_tableDict = createCtFieldDataDict(new_ctFieldTimeSeriesDataFiles)
        # create the table with new data
        new_CtField_DF = pd.DataFrame(new_tableDict)
        # fuse the two
        new_CtField_DF.index += existing_CtField_DF.shape[0]
        CtField_DF = pd.concat([existing_CtField_DF, new_CtField_DF])
    
    dateExemple = CtField_DF.loc[CtField_DF.index[0],'date']
    if re.match(dateFormatExcel, dateExemple):
        CtField_DF.loc[:,'date'] = CtField_DF.loc[:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])
    
    if save:
        saveName = fileName + '.csv'
        savePath = os.path.join(dataDir, saveName)
        CtField_DF.to_csv(savePath, sep=';')
        
    return(CtField_DF)

def getGlobalTable_ctField(fileName = 'Global_CtFieldData'):
    try:
        savePath = os.path.join(dataDir, (fileName + '.csv'))
        CtField_DF = pd.read_csv(savePath, sep=';')
        for c in CtField_DF.columns:
            if 'Unnamed' in c:
                CtField_DF = CtField_DF.drop([c], axis=1)
        print('Extracted a table with ' + str(CtField_DF.shape[0]) + ' lines and ' + str(CtField_DF.shape[1]) + ' columns.')
        
    except:
        print('No existing table found')
        
    dateExemple = CtField_DF.loc[CtField_DF.index[0],'date']
    if re.match(dateFormatExcel, dateExemple):
        print('dates corrected')
        CtField_DF.loc[:,'date'] = CtField_DF.loc[:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])
#         mecaDF['ManipID'] = mecaDF['ExpDay'] + '_' + mecaDF['CellName'].apply(lambda x: x.split('_')[0])
    return(CtField_DF)

In [ ]:
computeGlobalTable_ctField(task='updateExisting',save=False).head()

In [ ]:
getGlobalTable_ctField().head()

### Meca data

TBC

In [ ]:
def analyseTimeSeries_meca(tS_df):
    results = {}
    ### TBC
    return(results)

def createMecaDataDict(list_ctFieldFiles):
    tableDict = {}
    tableDict['date'], tableDict['cellName'], tableDict['cellID'] = [], [], []
#     tableDict[''], tableDict[''], tableDict[''] = [], [], []
#     tableDict[''], tableDict[''], tableDict[''] = [], [], []
#     tableDict[''] = []
# TBC
    for f in list_ctFieldFiles:
        split_f = f.split('_')
        tableDict['date'].append(split_f[0])
        tableDict['cellName'].append(split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        tableDict['cellID'].append(split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        tS_DataFilePath = os.path.join(timeSeriesDataDir, f)
        current_tS_df = pd.read_csv(tS_DataFilePath, ';')
        current_resultDict = analyseTimeSeries_meca(current_tS_df)
        for k in current_resultDict.keys():
            tableDict[k].append(current_resultDict[k])
    return(tableDict)

def computeGlobalTable_meca(task = 'fromScratch', fileName = 'Global_CtFieldData', save = True):
    mecaTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) \
                                  if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv") \
                                      and ('R40' in f))] # Change to allow different formats in the future
    print(ctFieldTimeSeriesDataFiles)
    if task == 'fromScratch':
        # create a dict containing the data
        tableDict = createMecaDataDict(mecaTimeSeriesDataFiles)
        # create the table
        meca_DF = pd.DataFrame(tableDict)
        
    elif task == 'updateExisting':
        # get existing table
        try:
            savePath = os.path.join(dataDir, (fileName + '.csv'))
            existing_meca_DF = pd.read_csv(savePath, sep=';')
        except:
            print('No existing table found')
        # find which of the time series files are new
        new_mecaTimeSeriesDataFiles = []
        for f in mecaTimeSeriesDataFiles:
            split_f = f.split('_')
            currentCellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
            if currentCellID not in existing_meca_DF.cellID:
                new_mecaTimeSeriesDataFiles.append(f)
        new_tableDict = createCtFieldDataDict(new_mecaTimeSeriesDataFiles)
        # create the table with new data
        new_meca_DF = pd.dataframe(tableDict)
        # fuse the two
        meca_DF = pd.concat([existing_meca_DF, new_meca_DF])
        
    if save:
        saveName = fileName + '.csv'
        savePath = os.path.join(dataDir, saveName)
        meca_DF.to_csv(savePath, sep=';')
        
    return(meca_DF)
              
def getGlobalTable_meca(fileName = 'Global_mecaData'):
    try:
        savePath = os.path.join(dataDir, (fileName + '.csv'))
        meca_DF = pd.read_csv(savePath, sep=';')
        print('Extracted a table with ' + str(meca_DF.shape[0]) + ' lines and ' + str(meca_DF.shape[1]) + ' columns.')
    except:
        print('No existing table found')
    for c in meca_DF.columns:
            if 'Unnamed' in c:
                meca_DF = meca_DF.drop([c], axis=1)
    if not ('manipID' in meca_DF.columns):
        meca_DF['manipID'] = meca_DF['ExpDay'] + '_' + meca_DF['CellID'].apply(lambda x: x.split('_')[0])
    dateExemple = meca_DF.loc[meca_DF.index[0],'ExpDay']
    if re.match(dateFormatExcel, dateExemple):
        print('bad date')
    return(meca_DF)

In [ ]:
getGlobalTable_meca().head()

### Fluorescence

In [ ]:
def getFluoData(save = False):
    # Getting the table
    fluoDataFile = 'FluoQuantification.csv'
    fluoDataFilePath = os.path.join(dataDir, fluoDataFile)
    fluoDF = pd.read_csv(fluoDataFilePath, sep=';',header=0)
    print('Extracted a table with ' + str(fluoDF.shape[0]) + ' lines and ' + str(fluoDF.shape[1]) + ' columns.')
    # Cleaning the table
    try:
        for c in fluoDF.columns:
            if 'Unnamed' in c:
                fluoDF = fluoDF.drop([c], axis=1)
        
    except:
        print('Unexpected bug with the cleaning step')

    if save:
        saveName = 'FluoQuantification.csv'
        savePath = os.path.join(dataDir, saveName)
        fluoDF.to_csv(savePath, sep=';')

    
    return(fluoDF)

In [ ]:
getFluoData().head()

### Data formatting and filtering

TBC

In [ ]:
mecaDataFile = 'Global_MecaData.csv'
mecaDataFilePath = os.path.join(dataDir, mecaDataFile)
mecaDF = pd.read_csv(mecaDataFilePath, sep=';')
print('Extracted a table with ' + str(mecaDF.shape[0]) + ' lines and ' + str(mecaDF.shape[1]) + ' columns.')

mecaDF = mecaDF.rename(columns={"CellID": "CellName", "CellName": "CellID"})

In [ ]:
# mecaDF

In [ ]:
experimentalDataFile = 'ExperimentalConditions.csv'
experimentalDataFilePath = os.path.join(experimentalDataDir, experimentalDataFile)
expConditionsDF = pd.read_csv(experimentalDataFilePath, sep=';',header=1)
print('Extracted a table with ' + str(expConditionsDF.shape[0]) + ' lines and ' + str(expConditionsDF.shape[1]) + ' columns.')

# Cleaning the table
try:
    expConditionsDF = expConditionsDF.convert_dtypes()

    listTextColumns = []
    for col in expConditionsDF.columns:
        if expConditionsDF[col].dtype == 'string':
            listTextColumns.append(col)

    expConditionsDF[listTextColumns] = expConditionsDF[listTextColumns].apply(lambda x: x.str.replace(',','.'))

    expConditionsDF['scale pixel per um'] = expConditionsDF['scale pixel per um'].astype(float)
    expConditionsDF['optical index correction'] = \
              expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[0]).astype(float) \
            / expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[1]).astype(float)
    expConditionsDF['magnetic field correction'] = expConditionsDF['magnetic field correction'].astype(float)
    expConditionsDF['with fluo images'] = expConditionsDF['with fluo images'].astype(bool)

    expConditionsDF['ramp field'] = \
    expConditionsDF['ramp field'].apply(lambda x: [x.split(';')[0], x.split(';')[1]] if not pd.isnull(x) else [])

except:
    print('Unexpected bug with the cleaning step')

In [ ]:
# expConditionsDF

In [ ]:
# Unused for now
cellDescriptionDataFile = 'CellDescription.csv'
cellDescriptionDataFilePath = os.path.join(experimentalDataDir, cellDescriptionDataFile)
cellDescriptionDF = pd.read_csv(cellDescriptionDataFilePath, ',')
print('Extracted a table with ' + str(cellDescriptionDF.shape[0]) + ' lines and ' + str(cellDescriptionDF.shape[1]) + ' columns.')

In [ ]:
mecaDF['ManipID'] = mecaDF['ExpDay'] + '_' + mecaDF['CellName'].apply(lambda x: x.split('_')[0])
expConditionsDF['ManipID'] = expConditionsDF['date'] + '_' + expConditionsDF['manip']

mainMecaDF = pd.merge(
    expConditionsDF,
    mecaDF,
    how="inner",
    on='ManipID',
#     left_on=None,
#     right_on=None,
#     left_index=False,
#     right_index=False,
#     sort=True,
#     suffixes=("_x", "_y"),
#     copy=True,
#     indicator=False,
#     validate=None,
)

In [ ]:
# pd.set_option('display.max_columns', None)
# mainMecaDF.head()

In [ ]:
pd.reset_option('max_columns')

#### * Data filtering

In [ ]:
# pd.set_option('display.max_columns', None)
# mainMecaDF.head()

In [ ]:
# pd.reset_option('max_columns')
# mainMecaDF.columns

In [ ]:
mainMecaDF_f = mainMecaDF.loc[(mainMecaDF["Validated"] == 1)]
# mainMecaDF_f

In [ ]:
%matplotlib inline
listCells = mainMecaDF_f['CellID'].drop_duplicates().astype('string').values
timeSeriesDict = {}
for cell in listCells:
    currentCell_TimeSeriesData = getCellTimeSeriesData(cell)
    timeSeriesDict[cell] = currentCell_TimeSeriesData
start, stop = 80, 100
fig, axes = plt.subplots((stop-start),1, figsize = (7,4*(stop-start)))
fig.tight_layout()
for k in range(start, stop):
    if k < len(listCells):
        currentCell_TimeSeriesData = timeSeriesDict[listCells[k]]
        T = currentCell_TimeSeriesData['T'].values
        idxCompression = currentCell_TimeSeriesData['idxCompression'].values
        D3 = currentCell_TimeSeriesData['D3'].values
        maskConstant = (idxCompression == 0)
        maskCompression = (idxCompression > 0)
        axes[k - start].plot(T, D3*1000-4503, 'k-', linewidth = 0.5)
        axes[k - start].plot(T[maskCompression], D3[maskCompression]*1000-4503, 'ro', markersize=2)
        axes[k - start].plot(T[maskConstant], D3[maskConstant]*1000-4503, 'co', markersize=2)
        axes[k - start].set_title(listCells[k])
        axes[k - start].set_xlabel('T (s)')
        axes[k - start].set_ylabel('D3 (µm)')

In [ ]:
addExcludedCell('21-01-18_M1_P1_C2', 'passive')
addExcludedCell('21-01-18_M1_P1_C3', 'passive')
addExcludedCell('21-01-18_M1_P1_C4', 'passive')
addExcludedCell('21-01-21_M3_P1_C4', 'passive')
addExcludedCell('21-01-21_M3_P1_C5', 'passive')
addExcludedCell('20-08-07_M1_P1_C6', 'too thick')
addExcludedCell('20-08-07_M1_P1_C62', 'too thick')

excludedCellsDict = getExcludedCells()
# # excludedMask = (mainMecaDF_f["CellID"].values not in excludedCellsDict.keys())
# # mainMecaDF_f = mainMecaDF_f.loc[(mainMecaDF_f["CellID"].values not in excludedCellsDict.keys())]
# for i in range(len(excludedCellsDict)):
#     print('a')
# mainMecaDF_f["CellID"].drop_duplicates().astype('string').values
excludedCellsDict

In [ ]:
# currentCell_TimeSeriesData

In [ ]:
mainMecaDF_GroupedPerCell = mainMecaDF_f.groupby('CellID')
mainMecaDF_DataPerCell = mainMecaDF_GroupedPerCell.agg({"EChadwick": np.median, "SurroundingThickness": np.median, "H0Chadwick" : np.median})
# mainMecaDF_GroupedPerCell.agg({"EChadwick": np.median, "D": lambda x: np.std(x, ddof=1)})
cols = ['date', 'manip', 'experimentType', 'drug', 'substrate',
       'objective magnification', 'scale pixel per um', 'objective immersion',
       'optical index correction', 'magnetic field correction', 'cell type',
       'cell subtype', 'bead type', 'bead diameter', 'normal field',
       'ramp field', 'compression duration', 'with fluo images', 'comments',
       'ManipID', 'ExpType', 'CellName', 'CellID']
mainMecaDF_DataPerCell.dropna(inplace = True)
mainMecaDF_DataPerCell = pd.merge(mainMecaDF_DataPerCell,
                                  mainMecaDF_f[cols].drop_duplicates(subset=['CellID']),
                                  how="inner",
                                  on='CellID',
                                  #     left_on='CellID',
                                  #     right_on='CellID',
                                  #     left_index=False,
                                  #     right_index=False,
                                  #     sort=True,
                                  #     suffixes=("_x", "_y"),
                                  #     copy=True,
                                  #     indicator=False,
                                  #     validate=None,
                                  )
# mainMecaDF_DataPerCell

In [ ]:
mainMecaDF_DataPerCell_Count = mainMecaDF_DataPerCell.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
mainMecaDF_DataPerCell_Count.loc[:, ['CellID']].rename(columns={'CellID' : 'Count'})

## Plots

### Functions

In [ ]:
Styles = {''}

In [ ]:
def D1Plot(data, CondCol=[],Parameters=[],Filters=[],Boxplot=True,AvgPerCell=False,cellID='cellID'):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
        
    NCond = len(CondCol)    
    if NCond == 1:
        CondCol = CondCol[0]
    elif NCond > 1:
        newColName = ''
        for i in range(NCond):
            newColName += CondCol[i]
            newColName += ' & '
        newColName = newColName[:-3]
        data_filtered[newColName] = ''
        for i in range(NCond):
            data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
            data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
        CondCol = newColName
    
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean)
        
    data_filtered.sort_values(CondCol, axis=0, ascending=False, inplace=True)
    
    NPlots = len(Parameters)
    Conditions = list(data_filtered[CondCol].unique())
    fig, ax = plt.subplots(1, NPlots, figsize = (4*NPlots*NCond,5))
    markerSize = 5
    if NPlots > 1:
        for k in range(NPlots):
#             vals = []
#             for i in range(len(Conditions)):
#                 vals.append(data_filtered[data_filtered[CondCol] == Conditions[i]][Parameters[k]])
            sns.swarmplot(x=CondCol, y=Parameters[k], data=data_filtered, ax=ax[k], 
                          size=markerSize, edgecolor='k',linewidth = 1)
            if Boxplot:
                sns.boxplot(x=CondCol, y=Parameters[k], data=data_filtered, ax=ax[k], 
                            color='w', linewidth = 2, width = 0.5, showfliers = False)
#                 data_filtered.boxplot(column=Parameters[k], by = CondCol, ax=ax[k],
#                             showfliers = False) # linewidth = 2, width = 0.5, 
            ax[k].set_ylabel(Parameters[k])
            ax[k].tick_params(axis='x', labelrotation = 10)
    else:
#         vals = []
#         for i in range(len(Conditions)):
#             vals.append(data_filtered[data_filtered[CondCol] == Conditions[i]][Parameters[0]])
        sns.swarmplot(x=CondCol, y=Parameters[0], data=data_filtered, ax=ax, 
                          size=markerSize, edgecolor='k',linewidth = 0.5)
        if Boxplot:
            sns.boxplot(x=CondCol, y=Parameters[0], data=data_filtered, ax=ax, 
                        color='w', linewidth = 2, width = 0.5, showfliers = False)
        ax.tick_params(axis='x', labelrotation = 25)
    return(fig, ax)

In [ ]:
def D1PlotInteractive(data, CondCol='',Parameters=[],Filters=[],AvgPerCell=False,cellID='cellID'):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
        
#     print(data_filtered[cellID])
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
    
#     return(data_filtered)
    
    NPlots = len(Parameters)
    Conditions = list(data_filtered[CondCol].unique())
    if NPlots > 1:
        plots = []
        NCond = len(Conditions)
        data_filtered['X'] = 0
        data_filtered['X_jitter'] = 0.
        dictTicks = {}
        for i in range(NCond):
            mask = data_filtered[CondCol] == Conditions[i]
            data_filtered.loc[mask, 'X'] = i+1
            dictTicks[i+1] = Conditions[i]
        for i in data_filtered.index:
            data_filtered.loc[i, 'X_jitter'] = data_filtered.loc[i, 'X'] + 0.4*(np.random.rand(1)[0]-0.5)
        source = ColumnDataSource(
            data=data_filtered[[cellID]+[CondCol]+Parameters+['X','X_jitter']]
        )        
        
        for k in range(NPlots):
            hover = HoverTool(
                tooltips=[
                    ('Cell ID', "@"+cellID),
                    (Parameters[k], "@"+Parameters[k]),
                ]
            )
            index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
            p = figure(plot_width=450, plot_height=500, tools=[hover], title="InteractivePlot") # 
            p.circle('X_jitter', Parameters[k], size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
            # Format
            p.x_range = Range1d(0, NCond+1)
            p.y_range = Range1d(min(0,1.1*np.min(data_filtered[Parameters[0]])), 1.1*np.max(data_filtered[Parameters[k]]))
            p.xaxis.ticker = [i for i in range(1,NCond+1)]
            p.xaxis.major_label_overrides = dictTicks
            p.xaxis.axis_label = CondCol
            p.xaxis.axis_label_text_font_size = '18pt'
            p.xaxis.major_label_text_font_size = '16pt'
            p.yaxis.axis_label = Parameters[k]
            p.yaxis.axis_label_text_font_size = '18pt'
            p.yaxis.major_label_text_font_size = '16pt'
            
            plots.append(p)
            
        p = gridplot(plots, ncols=2, toolbar_location=None)
        
        
    else:
        hover = HoverTool(
            tooltips=[
                ('Cell ID', "@"+cellID),
                (Parameters[0], "@"+Parameters[0]),
            ]
        )
        
        NCond = len(Conditions)
        data_filtered['X'] = 0
        data_filtered['X_jitter'] = 0.
        dictTicks = {}
        for i in range(NCond):
            mask = data_filtered[CondCol] == Conditions[i]
            data_filtered.loc[mask, 'X'] = i+1
            dictTicks[i+1] = Conditions[i]
        for i in data_filtered.index:
            data_filtered.loc[i, 'X_jitter'] = data_filtered.loc[i, 'X'] + 0.4*(np.random.rand(1)[0]-0.5)
        source = ColumnDataSource(
            data=data_filtered[[cellID]+[CondCol]+Parameters+['X','X_jitter']]
        )
        index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
        TOOLS = "hover,pan,box_zoom,wheel_zoom,reset,save,help"
        p = figure(plot_width=500, plot_height=500, tools=TOOLS, title="InteractivePlot") # 
        p.circle('X_jitter', Parameters[0], size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
        # Format
        p.x_range = Range1d(0, NCond+1)
        p.y_range = Range1d(min(0,1.1*np.min(data_filtered[Parameters[0]])), 1.1*np.max(data_filtered[Parameters[0]]))
        p.xaxis.ticker = [i for i in range(1,NCond+1)]
        p.xaxis.major_label_overrides = dictTicks
        p.xaxis.axis_label = CondCol
        p.xaxis.axis_label_text_font_size = '18pt'
        p.xaxis.major_label_text_font_size = '16pt'
        p.yaxis.axis_label = Parameters[0]
        p.yaxis.axis_label_text_font_size = '18pt'
        p.yaxis.major_label_text_font_size = '16pt'
    return(p)

In [ ]:
# data = GlobalTable_meca
# CondCol='drug'
# Parameters=['SurroundingThickness','EChadwick']
# Filters = [(GlobalTable_meca['Validated'] == 1)]
# AvgPerCell=True
# cellID='CellName'

# data_filtered = data
# for fltr in Filters:
#     data_filtered = data_filtered.loc[fltr]

# group = data_filtered.groupby(cellID)
# dictAggMean = getDictAggMean(data_filtered)
# data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
# data_filtered.reset_index(level=0, inplace=True)
# data_filtered=data_filtered[[cellID]+[CondCol]+Parameters]
# print(data_filtered)

In [ ]:
def D2Plot(data, XCol='',YCol='',CondCol='',Filters=[],AvgPerCell=False):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
    
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
        
    Conditions = list(data_filtered[CondCol].unique())
    fig, ax = plt.subplots(1, 1, figsize = (8,5))
    markerSize = 5
    for c in Conditions:
        ax.plot(data_filtered[data_filtered[CondCol] == c][XCol], 
                data_filtered[data_filtered[CondCol] == c][YCol],
               'o', markersize = markerSize, markeredgecolor='k',markeredgewidth = 1)
    ax.set_xlabel(XCol)
#     print(min(0,1.1*np.min(data_filtered[XCol])))
    ax.set_xlim([min(0,1.1*np.min(data_filtered[XCol])), 1.1*np.max(data_filtered[XCol])])
    ax.set_ylabel(YCol)
    ax.set_ylim([min(0,1.1*np.min(data_filtered[YCol])), 1.1*np.max(data_filtered[YCol])])
    return(fig, ax)

In [ ]:
def D2PlotInteractive(data, XCol='',YCol='',CondCol='',Filters=[], cellID='cellID',AvgPerCell=False):
    
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
        
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
    
    Conditions = list(data_filtered[CondCol].unique())

    NCond = len(Conditions)
    dictTicks = {}
    for i in range(NCond):
        dictTicks[i+1] = Conditions[i]
    
    source = ColumnDataSource(
        data=data_filtered[[cellID,CondCol,XCol,YCol]]
    )
    
    hover = HoverTool(
        tooltips=[
            ('Cell ID', "@"+cellID),
            (XCol, "@"+XCol),
            (YCol, "@"+YCol),
            (CondCol, "@"+CondCol),
        ]
    )
    
    index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
    TOOLS = "pan,box_zoom,wheel_zoom,reset,save,help"
    p = figure(plot_width=900, plot_height=500, tools=TOOLS, title="InteractivePlot",toolbar_location="below") # 
    p.circle(XCol, YCol, size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
    p.add_tools(hover)
    # Format
    p.x_range = Range1d(0, 1.1*np.max(data_filtered[XCol]))
    p.y_range = Range1d(0, 1.1*np.max(data_filtered[YCol]))
    p.xaxis.axis_label = XCol
    p.xaxis.axis_label_text_font_size = '18pt'
    p.xaxis.major_label_text_font_size = '16pt'
    p.yaxis.axis_label = YCol
    p.yaxis.axis_label_text_font_size = '18pt'
    p.yaxis.major_label_text_font_size = '16pt'
    return(p)

### Graphs

In [ ]:
pd.set_option('max_columns', None)
# pd.reset_option('max_columns')
pd.set_option('max_rows', None)
# pd.reset_option('max_rows')

In [ ]:
getGlobalTable_ctField().head()

In [ ]:
getFluoData().head()

In [ ]:
GlobalTable_ctField = getGlobalTable_ctField()
table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_ctField = pd.merge(table_ExpConditions, GlobalTable_ctField, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_ctField = pd.merge(GlobalTable_ctField, table_fluo, how="left", on='cellID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_ctField.shape[0]) + ' lines and ' + str(GlobalTable_ctField.shape[1]) + ' columns.')

In [ ]:
GlobalTable_ctField.head()

In [ ]:
GlobalTable_meca = getGlobalTable_meca()
table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_meca = pd.merge(table_ExpConditions, GlobalTable_meca, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_meca = pd.merge(GlobalTable_meca, table_fluo, how="left", left_on='CellName', right_on='cellID'
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_meca.shape[0]) + ' lines and ' + str(GlobalTable_meca.shape[1]) + ' columns.')

In [ ]:
GlobalTable_meca.tail()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True)]
fig, ax = D1Plot(GlobalTable_ctField, CondCol=['drug','substrate'],Parameters=['medianThickness','fluctuAmpli'],Filters=Filters)
fig.suptitle('3T3aSFL on patterns: Ct Field')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D1Plot(GlobalTable_meca, CondCol=['drug','substrate'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName')
fig.suptitle('3T3aSFL on diverse substrates: Ct Field')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
fig, ax = D1Plot(GlobalTable_meca, CondCol=['drug','substrate'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName')
fig.suptitle('3T3aSFL on patterns: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName')
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]

p = D1PlotInteractive(GlobalTable_ctField, CondCol='drug',Parameters=['medianThickness','fluctuAmpli'],Filters=Filters)
p.children[0][0].title.text = '3T3aSFL on patterns: Ct Field'
p.children[0][0].title.text_font_size = '16pt'
p.children[1][0].title.text = ''
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D1PlotInteractive(GlobalTable_meca, CondCol='drug',Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName')
p.children[0][0].title.text = '3T3aSFL on patterns: Compressions'
p.children[0][0].title.text_font_size = '14pt'
p.children[1][0].title.text = ''
show(p)

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = 'drug', Filters=Filters)
fig.show()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = 'drug', Filters=Filters)
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = 'drug', Filters=Filters, cellID = 'CellName')
show(p)

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True)]
p = D2PlotInteractive(GlobalTable_ctField, XCol='meanFluoPeakAmplitude',YCol='medianThickness',CondCol = 'drug', Filters=Filters, cellID = 'cellID',AvgPerCell=True)
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True)]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude',YCol='SurroundingThickness',CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = '3T3aSFL expressing linker: H(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL expressing linker: E(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude',YCol='SurroundingThickness',CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL-6FP expressing long linker: H(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL-6FP expressing long linker: E(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='SurroundingThickness', CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
P.title.text = 'aSFL expressing linker: E(fluo)'

show(p)

## Old Code

In [ ]:
# fig, axes = plt.subplots(1,2, figsize = (8, 5))
# axes[0].plot(np.ones(len(resDict['nodrug']['median'])), resDict['nodrug']['median'], 'co')
# axes[0].plot(2*np.ones(len(resDict['doxy']['median'])), resDict['doxy']['median'], 'ro')
# axes[0].set_xlim(0.5, 2.5)
# axes[0].set_ylabel('Median Thickness (nm)')
# axes[0].set_xticks([1,2])
# axes[0].set_xticklabels(['Control','Doxycylin'])
# axes[1].plot(np.ones(len(resDict['nodrug']['fluctu'])), resDict['nodrug']['fluctu'], 'co')
# axes[1].plot(2*np.ones(len(resDict['doxy']['fluctu'])), resDict['doxy']['fluctu'], 'ro')
# axes[1].set_xlim(0.5, 2.5)
# axes[1].set_ylabel('Thickness Fluctuations (nm)')
# axes[1].set_xticks([1,2])
# axes[1].set_xticklabels(['Control','Doxycylin'])
# fig.savefig("C://Users//JosephVermeil//Desktop//ActinCortexAnalysis//DataAnalysis//constantField.png")
# fig.show()

In [ ]:
# Old code
conditions = ['nodrug', 'doxy']
correspondance = {conditions[0] : 'M1', conditions[1] : 'M2'}
allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".txt"))]
dates = ['21-02-10']
resDict = {conditions[0] : {}, conditions[1] : {}}
for C in conditions:
    resDict[C]['accepted'] = []
    resDict[C]['rejected'] = []
    resDict[C]['median'] = []
    resDict[C]['fluctu'] = []
    for D in dates:
        for f in allTimeSeriesDataFiles:
            if correspondance[C] in f and D in f:
                split_f = f.split('_')
                cellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
                currentCellTS = getCellTimeSeriesData(cellID)
                D3 = currentCellTS.D3.values
                decile_1 = np.percentile(D3, 10)
                median = np.median(D3)
                decile_9 = np.percentile(D3, 90)
                if decile_1 < 0:
                    resDict[C]['rejected'].append(cellID)
                else:
                    resDict[C]['accepted'].append(cellID)
                    resDict[C]['median'].append(median)
                    resDict[C]['fluctu'].append(decile_9-decile_1)
#resDict

In [ ]:
GlobalTable_ctField = getGlobalTable_ctField()
table_ExpConditions = getExperimentalConditions()
GlobalTable_ctField = pd.merge(table_ExpConditions, GlobalTable_ctField, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)

# data=pd.DataFrame(dict(
#             x=[1, 2, 3, 4, 5],
#             y=[2, 5, 8, 2, 7],
#             desc=['A', 'A', 'C', 'd', 'E'],
#         ))

data = GlobalTable_ctField[['medianThickness','fluctuAmpli','cellID']]

source = ColumnDataSource(
        data=data
    )

# hover = HoverTool(
#         tooltips=[
#             ("index", "$index"),
#             ("(x,y)", "($x, $y)"),
#             ("desc", "@desc"),
#         ]
#     )

hover = HoverTool(
        tooltips=[
            ("medianThickness", "@medianThickness"),
            ("fluctuAmpli", "@fluctuAmpli"),
        ]
    )

p = figure(plot_width=300, plot_height=300, tools=[hover], title="Mouse over the dots")

p.circle('medianThickness', 'fluctuAmpli', size=20, source=data)

show(p)

data

In [ ]:
data=pd.DataFrame(dict(
            x=[1, 2, 3, 4, 5],
            y=[2, 5, 8, 2, 7],
            desc=['A', 'A', 'C', 'd', 'E'],
        ))

Conditions = list(data['desc'].unique())
NCond = len(Conditions)
data['X'] = 0
for i in range(NCond):
    mask = data['desc'] == Conditions[i]
    data.loc[mask, ['X']] = i+1
data.index = data.x
data = data.drop(['x'], axis = 1)

data.reset_index(level=0, inplace=True)
data